# GoLLIE for Medical Entity Extraction

### Import requeriments
See the requeriments.txt file in the main directory to install the required dependencies


In [1]:
import sys
sys.path.append("../") # Add the GoLLIE base directory to sys path

In [2]:
import rich 
import logging
from src.model.load_model import load_model
import black
import inspect
from jinja2 import Template as jinja2Template
import tempfile
from src.tasks.utils_typing import AnnotationList
logging.basicConfig(level=logging.INFO)
from typing import Dict, List, Type

## Load GoLLIE

Load GOLLIE-7B from the huggingface-hub.
Use the AutoModelForCausalLM.from_pretrained function if you prefer it. However, creators provide a handy load_model function with many functionalities already implemented that will assist you in reproducing our results.

Please note that setting use_flash_attention=True is mandatory. Our flash attention implementation has small numerical differences compared to the attention implementation in Huggingface. Using use_flash_attention=False will result in the model producing inferior results. Flash attention requires an available CUDA GPU. Running GOLLIE pre-trained models on a CPU is not supported.

- Set force_auto_device_map=True to automatically load the model on available GPUs.
- Set quantization=4 if the model doesn't fit in your GPU memory.

In [3]:
#Use the custom load_model for loading GoLLIE
model, tokenizer = load_model(
    inference=True,
    model_weights_name_or_path="HiTZ/GoLLIE-7B",
    quantization=None,
    use_lora=False,
    force_auto_device_map=True,
    use_flash_attention=True,
    torch_dtype="bfloat16"
)

INFO:root:Loading model model from HiTZ/GoLLIE-7B
INFO:root:We will load the model using the following device map: auto and max_memory: None
INFO:root:Loading model with dtype: torch.bfloat16


>>>> Flash Attention installed
>>>> Flash RoPE installed


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

INFO:root:Model dtype: torch.bfloat16
INFO:root:Total model memory footprint: 13477.101762 MB


## Define the guidelines

First, we will define the labels and guidelines for the task. We will represent them as Python classes.

The following guidelines have been defined for this example. They were not part of the pre-training dataset. Therefore, we will run GOLLIE in zero-shot settings using unseen labels.

We will use the `Generic` class, which is a versatile class that allows for the implementation of any task you want. However, since the model has never seen the Generic label during training, we will rename it to Template, which is recognized by the model (as it was used in the Tacred dataset).

We will define several classes: `Illness`, `Medication`, `Procedure`, `VitalSigns`, `LabResult`, `PatientDemographics`, `HospitalizationData`. Each class will have a definition and a set of slots that the model needs to fill. Each slot also requires a type definition and a short description, which can include examples. For instance, for the `Illness` class, we define three slots:

- The `mention`, which will be the name of the Ilness of the patient and should be a string.
- The `chronic`, which will be a boolean indicating if the illness is chronic or not.
- The `diagnosed_date`, which will be a date indicating when the illness was diagnosed and should be a string.
- The `associated_symptoms`, which is defined as a list of symptoms. Therefore, GoLLIE will fill this slot with a list of strings.


In [ ]:
from typing import List

from src.tasks.utils_typing import dataclass
from src.tasks.utils_typing import Generic as Template

"""
Entity definitions
"""


@dataclass
class Illness:
    """
    Represents illnesses or diseases diagnosed or discussed in medical documents.
    
    Attributes:
        mention (str): Exact text mentioning the illness.
        chronic (bool): Indicates whether the illness is chronic. Defaults to False.
        diagnosed_date (str): The date on which the illness was diagnosed, if available.
        associated_symptoms (List[str]): List of primary symptoms associated with the illness.
    """
    mention: str
    """
    The name of the illness.
    Such as: "COVID-19", "Diabetes", "Cancer"
    """
    chronic: bool = False
    diagnosed_date: str = ""
    associated_symptoms: List[str] = field(default_factory=list)


@dataclass
class Launcher(Template):
    """Refers to a vehicle designed primarily to transport payloads from the Earth's 
    surface to space. Launchers can carry various payloads, including satellites, 
    crewed spacecraft, and cargo, into various orbits or even beyond Earth's orbit. 
    They are usually multi-stage vehicles that use rocket engines for propulsion."""

    mention: str  
    """
    The name of the launcher vehicle. 
    Such as: "Sturn V", "Atlas V", "Soyuz", "Ariane 5"
    """
    space_company: str # The company that operates the launcher. Such as: "Blue origin", "ESA", "Boeing", "ISRO", "Northrop Grumman", "Arianespace"
    crew: List[str] # Names of the crew members boarding the Launcher. Such as: "Neil Armstrong", "Michael Collins", "Buzz Aldrin"
    

@dataclass
class Mission(Template):
    """Any planned or accomplished journey beyond Earth's atmosphere with specific objectives, 
    either crewed or uncrewed. It includes missions to satellites, the International 
    Space Station (ISS), other celestial bodies, and deep space."""
    
    mention: str
    """
    The name of the mission. 
    Such as: "Apollo 11", "Artemis", "Mercury"
    """
    date: str # The start date of the mission
    departure: str # The place from which the vehicle will be launched. Such as: "Florida", "Houston", "French Guiana"
    destination: str # The place or planet to which the launcher will be sent. Such as "Moon", "low-orbit", "Saturn"


ENTITY_DEFINITIONS: List[Template] = [
    Launcher,
    Mission,
]
    
if __name__ == "__main__":
    cell_txt = In[-1]